In [2]:
import cloudComPy as cc

ModuleNotFoundError: No module named 'cloudComPy'

In [7]:
0.014/10

0.0014

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
from simpleicp import SimpleICP, PointCloud
def compute_similarity_transform(src_points, dst_points):
    """
    Compute similarity transformation matrix based on corresponding points.
    
    Parameters:
        src_points (ndarray): Source point cloud, Nx3 array.
        dst_points (ndarray): Destination point cloud, Nx3 array.
        
    Returns:
        similarity_matrix (ndarray): 4x4 similarity transformation matrix.
    """
    # Compute centroids
    src_centroid = np.mean(src_points, axis=0)
    dst_centroid = np.mean(dst_points, axis=0)
    
    # Center the point clouds
    src_centered = src_points - src_centroid
    dst_centered = dst_points - dst_centroid
    
    # Compute scale
    scale = np.linalg.norm(dst_centered) / np.linalg.norm(src_centered)
    
    # Compute rotation using SVD
    H = np.dot(src_centered.T, dst_centered)
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)
    
    # Construct similarity transformation matrix
    similarity_matrix = np.eye(4)
    similarity_matrix[:3, :3] = R * scale
    similarity_matrix[:3, 3] = dst_centroid - np.dot(similarity_matrix[:3, :3], src_centroid)
    
    return similarity_matrix

def apply_similarity_transform(points, similarity_matrix):
    """
    Apply similarity transformation to a set of points.
    
    Parameters:
        points (ndarray): Point cloud, Nx3 array.
        similarity_matrix (ndarray): 4x4 similarity transformation matrix.
        
    Returns:
        transformed_points (ndarray): Transformed point cloud, Nx3 array.
    """
    homogeneous_points = np.hstack([points, np.ones((points.shape[0], 1))])
    transformed_points = np.dot(homogeneous_points, similarity_matrix.T)[:, :3]
    return transformed_points

In [1]:
distTest1 = cc.loadPointCloud("distTest1.bin")
distTest2 = cc.loadPointCloud("distTest2.bin")

NameError: name 'cc' is not defined

In [4]:

distTest1.deleteAllScalarFields()

In [5]:

bestOctreeLevel = cc.DistanceComputationTools.determineBestOctreeLevel(distTest1, None, distTest2)
params = cc.Cloud2CloudDistancesComputationParams()
params.maxThreadCount = 12
params.octreeLevel = bestOctreeLevel
stats = cc.DistanceComputationTools.computeCloud2CloudDistances(distTest1, distTest2, params)

In [6]:
dic = distTest1.getScalarFieldDic()
sf = distTest1.getScalarField(dic['C2C absolute distances'])

In [7]:
distances = sf.toNpArray()

In [8]:
dic

{'Approx. distances': 0, 'C2C absolute distances': 1}

In [9]:
distTest1.setCurrentDisplayedScalarField(1)

In [10]:
distTest1.getCurrentDisplayedScalarField()

In [11]:
cola = distTest1.convertCurrentScalarFieldToColors()

In [12]:
cola = distTest1.colorsToNpArray()

In [13]:
points = distTest1.toNpArrayCopy()

In [14]:
cola.shape

(40964, 4)

In [66]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import numpy as np

# Create sample data
np.random.seed(0)
num_points = 100
x = np.random.randn(num_points)
y = np.random.randn(num_points)
z = np.random.randn(num_points)
color_data = cola  # Random color data in (r, g, b, alpha) format

# Initialize Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(
        id='3d-scatter-plot',
        figure={
            'data': [
                go.Scatter3d(
                    x=points[:,0],
                    y=points[:,1],
                    z=points[:,2],
                    mode='markers',
                    marker=dict(
                        size=1,
                        color=distances,  # Set color based on color data
                        opacity=0.8,
                        colorscale='Viridis',  # Set color scale
                        colorbar=dict(title='Color Legend')  # Add color legend
                    )
                )
            ],
            'layout': go.Layout(
                title='3D Scatter Plot',
                scene=dict(
                    xaxis=dict(title='X'),
                    yaxis=dict(title='Y'),
                    zaxis=dict(title='Z')
                )
            )
        }
    )
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

In [12]:
stats = cc.DistanceComputationTools.computeApproxCloud2CloudDistance(distTest1,distTest2)

In [13]:
stats

[0.0,
 0.008171586319804192,
 0.000772103201597929,
 9.77202375906927e-07,
 0.00027735461480915546]

In [15]:
distTest1.setCurrentDisplayedScalarField(1)

In [16]:
distTest1.convertCurrentScalarFieldToColors()

True

In [19]:
distTest1.showColors(True)

In [23]:

import cloudComPy.M3C2
params = cc.M3C2.M3C2guessParamsToFile([distTest1,distTest2],"m3c2params.bin")

TypeError: M3C2guessParamsToFile(): incompatible function arguments. The following argument types are supported:
    1. (arg0: List[_cloudComPy.ccHObject], arg1: QString, arg2: bool) -> bool

Invoked with: [<_cloudComPy.ccPointCloud object at 0x000001A255272CF0>, <_cloudComPy.ccPointCloud object at 0x000001A2552E9C30>], 'm3c2params.bin'

In [63]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import numpy as np

# Create sample data
np.random.seed(0)
num_points = 100
x = np.random.randn(num_points)
y = np.random.randn(num_points)
z = np.random.randn(num_points)
distances = np.random.rand(num_points)  # Random distances

# Define color scale (Viridis) for distances
color_scale = 'Viridis'

# Initialize Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    dcc.Graph(
        id='3d-scatter-plot',
        figure={
            'data': [
                go.Scatter3d(
                    x=x,
                    y=y,
                    z=z,
                    mode='markers',
                    marker=dict(
                        size=5,
                        color=distances,  # Set color based on distances
                        colorscale=color_scale,  # Set color scale
                        opacity=0.8,
                        colorbar=dict(title='Distance', tickvals=[0, 0.5, 1], ticktext=['Low', 'Medium', 'High'])  # Add color legend
                    )
                )
            ],
            'layout': go.Layout(
                title='3D Scatter Plot',
                scene=dict(
                    xaxis=dict(title='X'),
                    yaxis=dict(title='Y'),
                    zaxis=dict(title='Z')
                )
            )
        }
    )
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)